In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

import helpers as hp

%load_ext autoreload
%autoreload 2


# Download rent data

In [2]:
base_url = 'http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/'
file_names = [
    'detailed-lodged-bonds.csv',
    'detailed-mean-rents.csv',
    'detailed-geo-mean-rents.csv',
]

for file_name in file_names:
    url = base_url + file_name
    print("Getting and saving", url)
    r = requests.get(url)
    path = hp.DATA_DIR/'collected'/file_name
    with path.open('w') as tgt:
        tgt.write(r.text)


Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-lodged-bonds.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-mean-rents.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-geo-mean-rents.csv


# Prepare rent data

In [5]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    hp.DATA_DIR/'collected'/'detailed-lodged-bonds.csv',
    hp.DATA_DIR/'collected'/'detailed-mean-rents.csv',
    hp.DATA_DIR/'collected'/'detailed-geo-mean-rents.csv',
]
names = [
    'rent_count', 
    'rent_mean', 
    'rent_geo_mean',
]
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
242,500202,Flat or Apartment,1,1995-03-01,8.0,82.0,82.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
262,500202,Flat or Apartment,1,1996-03-01,5.0,79.0,79.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
265,500202,House,3,1996-03-01,5.0,123.0,118.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
290,500202,House,3,1997-06-01,5.0,154.0,152.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
315,500202,House,3,1998-09-01,7.0,139.0,137.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo


In [6]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2017-12-01', '2018-03-01']

# Explore rents

In [7]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [8]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2017-03-01')
agg_rents.head()

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,296.0,211.266524,214.537162,Christchurch City
1,Addington,2,Canterbury,166.0,326.101925,331.415663,Christchurch City
2,Addington,3,Canterbury,153.0,402.602610,410.366013,Christchurch City
3,Addington,4,Canterbury,10.0,466.527598,473.000000,Christchurch City
4,Addington,5+,Canterbury,0.0,NaN,NaN,Christchurch City


In [10]:
f = hp.aggregate_rents(rents, '2017-03-01', groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
f[cond]

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,au2001,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
286,505300,1,Auckland,0.0,NaN,NaN,Rodney District
287,505300,2,Auckland,11.0,370.512838,373.636364,Rodney District
288,505300,3,Auckland,42.0,408.165092,411.595238,Rodney District
289,505300,4,Auckland,20.0,454.897926,456.100000,Rodney District
290,505300,5+,Auckland,0.0,NaN,NaN,Rodney District
291,505400,1,Auckland,0.0,NaN,NaN,Rodney District
292,505400,2,Auckland,0.0,NaN,NaN,Rodney District
293,505400,3,Auckland,0.0,NaN,NaN,Rodney District
294,505400,4,Auckland,0.0,NaN,NaN,Rodney District
295,505500,1,Auckland,12.0,284.124064,286.250000,Rodney District


In [11]:
# What fraction of rental data do we have by #bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2017-03-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


census area units
  #bedrooms  hit_frac
0         1  0.414330
1         2  0.704348
2         3  0.868946
3         4  0.665706
4        5+  0.175719
rental area units
  #bedrooms  hit_frac
0         1  0.787879
1         2  0.979798
2         3  1.000000
3         4  0.929293
4        5+  0.353535


# Select latest two quarters and slice into regional chunks

In [12]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents[rents['quarter'] >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents[rents['region'] == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2017-12-01' '2018-03-01'] #rows = 5508
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2017-12-01' '2018-03-01'] #rows = 2530
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2017-12-01' '2018-03-01'] #rows = 2410
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [13]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


** auckland 
 {'Glen Eden': {'4': 551.0, '3': 473.0, '1': None, '2': 419.0}, 'Waterview/Avondale Heights': {'4': 691.0, '3': 535.0, '1': 315.0, '2': 454.0}, 'Mangere Bridge/Airport': {'4': 603.0, '3': 521.0, '1': 348.0, '2': 471.0}, 'Pukekohe/Tuakau': {'4': 535.0, '3': 459.0, '1': 338.0, '2': 389.0}, 'Balmoral': {'4': 855.0, '3': 679.0, '1': 379.0, '2': 479.0}, 'Newmarket': {'4': None, '3': 757.0, '1': 408.0, '2': 563.0}, 'Manurewa North': {'4': 555.0, '3': 489.0, '1': 319.0, '2': 416.0}, 'Manukau Heights and Manurewa Heights': {'4': 647.0, '3': 571.0, '1': None, '2': 447.0}, 'Onehunga/Oranga/Te Papapa': {'4': 717.0, '3': 608.0, '1': 250.0, '2': 469.0}, 'Weymouth': {'4': 579.0, '3': 494.0, '1': None, '2': None}, 'Hillcrest/Northcote': {'4': 680.0, '3': 579.0, '1': None, '2': 448.0}, 'Browns Bay': {'4': 711.0, '3': 592.0, '1': None, '2': 506.0}, 'St Lukes/Sandringham': {'4': 772.0, '3': 662.0, '1': 354.0, '2': 480.0}, 'Remuera': {'4': 1077.0, '3': 771.0, '1': None, '2': 602.0}, 'Otahuhu

** wellington 
 {'Epuni/Avalon': {'4': 675.0, '3': 481.0, '1': 177.0, '2': 413.0}, 'Porirua East/Waitangirua': {'4': 457.0, '3': 334.0, '1': None, '2': 328.0}, 'Northland': {'4': 585.0, '3': 617.0, '1': 328.0, '2': 493.0}, 'Mt Cook': {'4': 674.0, '3': 626.0, '1': 240.0, '2': 511.0}, 'Eastern Bays': {'4': 717.0, '3': 588.0, '1': None, '2': None}, 'Mt Victoria/Roseneath': {'4': 709.0, '3': 642.0, '1': 319.0, '2': 534.0}, 'Totara Park/Maoribank/Te Marua': {'4': None, '3': 444.0, '1': None, '2': None}, 'Stokes Valley': {'4': None, '3': 417.0, '1': None, '2': None}, 'Petone West/Alicetown': {'4': None, '3': 534.0, '1': None, '2': 408.0}, 'Brooklyn': {'4': 702.0, '3': 629.0, '1': 340.0, '2': 475.0}, 'Masterton': {'4': None, '3': 314.0, '1': 193.0, '2': 265.0}, 'Ngaio/Kaiwharawhara/Wilton': {'4': 743.0, '3': 478.0, '1': 321.0, '2': 405.0}, 'Papakowhai/Whitby/Pauatahanui': {'4': 654.0, '3': 479.0, '1': None, '2': None}, 'Hataitai': {'4': 733.0, '3': 613.0, '1': 283.0, '2': 439.0}, 'Wadestown/T